# 第12章：Agent with Tools（工具增强）

## 学习目标

本章将学习：
1. Agent工具管理的高级技巧
2. 工具错误处理与自动重试
3. Human-in-the-loop（人工审批）
4. Tool artifacts的使用
5. 工具调用链与多步推理
6. Middleware自定义工具行为
7. 实战：搜索+计算Agent
8. 实战：数据处理Agent

## 为什么需要工具增强？

基础Agent已经能调用工具，但生产环境需要：

1. **可靠性**：工具失败时自动重试
2. **安全性**：敏感操作需人工审批
3. **可观测性**：监控工具调用和性能
4. **灵活性**：动态选择和过滤工具
5. **数据管理**：分离给LLM的内容和给应用的元数据（artifacts）

---

## 核心概念

### Agent工具调用流程

```
User Query
    ↓
  Model  ← Middleware可以拦截和修改
    ↓
Tool Selection
    ↓
Tool Execution ← 可能失败，需重试/审批
    ↓
Tool Result (content + artifact)
    ↓
  Model
    ↓
Final Answer
```

In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. 工具错误处理与自动重试

生产环境中工具可能失败（网络问题、API限流、超时等），需要：

1. **自动重试**：使用 `ToolRetryMiddleware`
2. **自定义错误处理**：通过 `wrap_tool_call` 拦截错误
3. **让LLM感知错误**：返回错误信息而非抛出异常

---

### 1.1 基础错误演示

In [2]:
print("【演示：工具失败场景】")
print()

# 创建一个会失败的工具
call_count = 0

@tool
def unstable_api(query: str) -> str:
    """调用不稳定的API（模拟）"""
    global call_count
    call_count += 1
    
    # 前2次调用失败，第3次成功
    if call_count < 3:
        raise Exception(f"API调用失败 (尝试 {call_count}/3)")
    
    return f"成功获取'{query}'的数据"

# 不使用错误处理的Agent
agent_no_retry = create_agent(
    model=model,
    tools=[unstable_api],
    system_prompt="使用unstable_api工具查询信息。"
)

print("【测试1：无错误处理】")
try:
    response = agent_no_retry.invoke({
        "messages": [{"role": "user", "content": "查询天气"}]
    })
    print(f"结果: {response['messages'][-1].content}")
except Exception as e:
    print(f"失败: {str(e)}")
    print("说明: 工具失败导致整个Agent执行中断")

# 重置计数
call_count = 0
print(f"\n✓ 无错误处理时，工具失败会导致Agent停止")

【演示：工具失败场景】

【测试1：无错误处理】
失败: API调用失败 (尝试 1/3)
说明: 工具失败导致整个Agent执行中断

✓ 无错误处理时，工具失败会导致Agent停止


### 1.2 使用ToolRetryMiddleware自动重试

In [3]:
print("【演示：自动重试机制】")
print()

from langchain.agents.middleware import ToolRetryMiddleware

# 重置计数
call_count = 0

# 使用ToolRetryMiddleware
agent_with_retry = create_agent(
    model=model,
    tools=[unstable_api],
    system_prompt="使用unstable_api工具查询信息。",
    middleware=[
        ToolRetryMiddleware(
            max_retries=2,          # 最多重试2次
            backoff_factor=2.0,     # 指数退避系数
            initial_delay=0.5,      # 初始延迟（秒）
            on_failure="return_message"  # 失败后返回错误消息而非抛出异常
        )
    ]
)

print("【测试2：使用自动重试】")
response = agent_with_retry.invoke({
    "messages": [{"role": "user", "content": "查询天气"}]
})

print(f"工具调用次数: {call_count}")
print(f"最终结果: {response['messages'][-1].content}")
print()

【演示：自动重试机制】

【测试2：使用自动重试】


/tmp/ipykernel_1645757/1354626572.py:15: DeprecationWarning: on_failure='return_message' is deprecated and will be removed in a future version. Use on_failure='continue' instead.
  ToolRetryMiddleware(


工具调用次数: 3
最终结果: 请问您想查询哪个城市的天气？



### 1.3 自定义错误处理Middleware

In [4]:
print("【演示：自定义错误处理】")
print()

from langchain.agents.middleware import wrap_tool_call
from langchain.tools.tool_node import ToolCallRequest
from langchain.messages import ToolMessage

@wrap_tool_call
def custom_error_handler(request: ToolCallRequest, handler):
    """自定义工具错误处理"""
    try:
        # 执行工具
        result = handler(request)
        print(f"✓ 工具 {request.tool_call['name']} 执行成功")
        return result
    except Exception as e:
        # 捕获错误，返回友好的错误消息给LLM
        error_msg = f"工具执行失败: {str(e)}. 请换一个方式尝试。"
        print(f"✗ 工具 {request.tool_call['name']} 执行失败: {e}")
        
        # 返回ToolMessage而非抛出异常，让LLM看到错误并调整策略
        return ToolMessage(
            content=error_msg,
            tool_call_id=request.tool_call['id']
        )

# 创建一个会失败的工具（不重试）
@tool
def always_fail_tool(query: str) -> str:
    """总是失败的工具"""
    raise ValueError("此工具暂时不可用")

@tool
def backup_tool(query: str) -> str:
    """备用工具"""
    return f"使用备用方案获取'{query}'的信息"

# 使用自定义错误处理
agent_custom_error = create_agent(
    model=model,
    tools=[always_fail_tool, backup_tool],
    system_prompt="优先使用always_fail_tool，如果失败则使用backup_tool。",
    middleware=[custom_error_handler]
)

print("【测试3：自定义错误处理】")
response = agent_custom_error.invoke({
    "messages": [{"role": "user", "content": "查询股票价格"}]
})

print(f"\n最终答案: {response['messages'][-1].content}")
print()
print("说明: LLM看到第一个工具失败后，自动切换到备用工具")

【演示：自定义错误处理】

【测试3：自定义错误处理】
✗ 工具 always_fail_tool 执行失败: 此工具暂时不可用
✓ 工具 backup_tool 执行成功

最终答案: 尝试使用总是失败的工具查询股票价格失败，现已使用备用工具为您获取了相关信息。请问您需要查询具体哪只股票的价格？

说明: LLM看到第一个工具失败后，自动切换到备用工具


## 2. Tool Artifacts（工具元数据）

**问题**：工具返回的数据可能包含两部分：
- **content**：给LLM看的文本描述
- **artifact**：给应用程序用的结构化数据（如文档ID、原始数据、元数据）

**解决方案**：使用 `response_format="content_and_artifact"`

---

### 为什么需要Artifacts？

| 场景 | content（给LLM） | artifact（给应用） |
|------|----------------|----------------|
| 检索文档 | "找到2篇相关文章..." | 完整Document对象（含metadata） |
| 查询数据库 | "共3条记录..." | 原始数据列表 |
| 调用API | "API返回成功" | 完整响应JSON |

**关键点**：artifact不会发送给LLM，不占用token，但应用可以访问

In [5]:
print("【演示：Tool Artifacts】")
print()

# 模拟数据库
user_database = [
    {"id": 1, "name": "Alice", "email": "alice@example.com", "age": 28},
    {"id": 2, "name": "Bob", "email": "bob@example.com", "age": 35},
    {"id": 3, "name": "Charlie", "email": "charlie@example.com", "age": 42}
]

@tool(response_format="content_and_artifact")
def search_users(query: str):
    """搜索用户数据库"""
    results = [u for u in user_database if query.lower() in u["name"].lower()]
    
    # content: 给LLM看的简洁描述
    content = f"找到 {len(results)} 个用户：" + ", ".join(u["name"] for u in results)
    
    # artifact: 给应用用的完整数据
    artifact = {
        "total": len(results),
        "users": results,
        "query": query
    }
    
    return content, artifact

# 创建Agent
agent_with_artifact = create_agent(
    model=model,
    tools=[search_users],
    system_prompt="使用search_users工具查询用户信息。"
)

# 调用Agent
response = agent_with_artifact.invoke({
    "messages": [{"role": "user", "content": "查找名字中包含'li'的用户"}]
})

# 提取artifact
print("【LLM的最终回答】")
print(response['messages'][-1].content)
print()

print("【从ToolMessage中提取artifact】")
for msg in response['messages']:
    if msg.__class__.__name__ == 'ToolMessage' and hasattr(msg, 'artifact'):
        print(f"工具名称: {msg.name}")
        print(f"Content (给LLM): {msg.content}")
        print(f"Artifact (给应用): {msg.artifact}")
        print()
        print("应用可以使用artifact中的完整数据：")
        for user in msg.artifact['users']:
            print(f"  - ID: {user['id']}, Email: {user['email']}, Age: {user['age']}")

print()
print("说明: LLM只看到简洁的content，应用可以访问artifact中的完整结构化数据")

【演示：Tool Artifacts】

【LLM的最终回答】
名字中包含'li'的用户有：Alice 和 Charlie。需要我帮您查询他们的详细信息吗？

【从ToolMessage中提取artifact】
工具名称: search_users
Content (给LLM): 找到 2 个用户：Alice, Charlie
Artifact (给应用): {'total': 2, 'users': [{'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'age': 28}, {'id': 3, 'name': 'Charlie', 'email': 'charlie@example.com', 'age': 42}], 'query': 'li'}

应用可以使用artifact中的完整数据：
  - ID: 1, Email: alice@example.com, Age: 28
  - ID: 3, Email: charlie@example.com, Age: 42

说明: LLM只看到简洁的content，应用可以访问artifact中的完整结构化数据


## 3. Human-in-the-loop（人工审批）

某些操作风险较高，需要人工审批后才能执行：

- 删除数据
- 发送邮件
- 执行SQL语句
- 调用付费API

---

### 工作流程

```
User: "删除用户Alice的账户"
    ↓
Agent决定调用 delete_user
    ↓
【中断执行】等待人工审批
    ↓
人工决策：
  - Approve（批准）→ 执行工具
  - Edit（修改参数）→ 修改后执行
  - Reject（拒绝）→ 取消执行
    ↓
继续Agent执行
```

---

### 概念说明

LangChain v1的Human-in-the-loop基于LangGraph的：
1. **interrupt机制**：在工具执行前暂停
2. **checkpointer**：保存Agent状态
3. **update_state**：应用人工决策后继续

**注意**：完整实现需要LangGraph的持久化（后续章节学习），这里演示简化的监控方式

In [6]:
print("【演示：简化的人工审批（监控模式）】")
print()

# 定义敏感工具
@tool
def delete_user(user_id: int) -> str:
    """删除用户账户（危险操作）"""
    return f"用户 {user_id} 已被删除"

@tool
def query_user(user_id: int) -> str:
    """查询用户信息（安全操作）"""
    return f"用户 {user_id} 的信息：姓名=Alice, 邮箱=alice@example.com"

# 创建审批中间件
@wrap_tool_call
def approval_middleware(request: ToolCallRequest, handler):
    """敏感操作需要人工审批"""
    tool_name = request.tool_call['name']
    
    # 定义需要审批的工具
    sensitive_tools = ['delete_user']
    
    if tool_name in sensitive_tools:
        print(f"\n⚠️  敏感操作检测: {tool_name}")
        print(f"   参数: {request.tool_call['args']}")
        print(f"   等待人工审批...")
        
        # 模拟人工审批（实际应用中这里会暂停等待真实的人工决策）
        approval = input("   批准此操作？(yes/no): ").strip().lower()
        
        if approval == 'yes':
            print("   ✓ 操作已批准，执行中...")
            return handler(request)
        else:
            print("   ✗ 操作被拒绝")
            return ToolMessage(
                content=f"操作被人工拒绝: {tool_name}不允许执行",
                tool_call_id=request.tool_call['id']
            )
    else:
        # 非敏感操作直接执行
        print(f"✓ 安全操作: {tool_name} 直接执行")
        return handler(request)

# 创建Agent
agent_with_approval = create_agent(
    model=model,
    tools=[delete_user, query_user],
    system_prompt="根据用户请求执行相应操作。",
    middleware=[approval_middleware]
)

print("【测试场景1：安全操作】")
response1 = agent_with_approval.invoke({
    "messages": [{"role": "user", "content": "查询用户ID为123的信息"}]
})
print(f"结果: {response1['messages'][-1].content}")

print("\n" + "="*60)
print("【测试场景2：敏感操作（需手动输入yes或no）】")
# 取消注释下面代码以测试审批流程
response2 = agent_with_approval.invoke({
    "messages": [{"role": "user", "content": "删除用户ID为456的账户"}]
})
print(f"结果: {response2['messages'][-1].content}")

print("\n说明: 敏感操作会触发审批流程，安全操作直接执行")

【演示：简化的人工审批（监控模式）】

【测试场景1：安全操作】
✓ 安全操作: query_user 直接执行
结果: 用户ID为123的信息如下：
姓名：Alice
邮箱：alice@example.com

请问还需要查询其他用户的信息吗？

【测试场景2：敏感操作（需手动输入yes或no）】

⚠️  敏感操作检测: delete_user
   参数: {'user_id': 456}
   等待人工审批...
   ✓ 操作已批准，执行中...
结果: 用户ID为456的账户已被成功删除。还有其他需要帮助的吗？

说明: 敏感操作会触发审批流程，安全操作直接执行


## 4. 工具调用链与多步推理

Agent的强大之处在于能够：
1. **串行调用**：一个工具的输出作为下一个工具的输入
2. **并行调用**：同时调用多个独立的工具
3. **自适应推理**：根据中间结果决定下一步

---

### 典型场景

**场景1：数据处理链**
```
查询数据库 → 数据分析 → 生成报告
```

**场景2：信息聚合**
```
并行查询: [天气API + 新闻API + 股票API] → 综合摘要
```

In [7]:
print("【演示：工具调用链】")
print()

# 定义工具链
@tool
def fetch_sales_data(product: str) -> str:
    """获取产品的销售数据"""
    data = {"laptop": "100台", "phone": "250台", "tablet": "75台"}
    return f"{product}的销售量: {data.get(product, '无数据')}"

@tool
def calculate_revenue(sales_data: str) -> str:
    """根据销售数据计算收入"""
    # 简单模拟：提取数量并计算
    import re
    match = re.search(r'(\d+)台', sales_data)
    if match:
        quantity = int(match.group(1))
        unit_price = 5000  # 假设单价
        revenue = quantity * unit_price
        return f"计算结果: 收入为 {revenue:,} 元"
    return "无法计算"

@tool
def generate_report(revenue_info: str) -> str:
    """生成销售报告"""
    return f"【销售报告】\n{revenue_info}\n报告生成时间: 2024-01-15"

# 创建Agent
agent_chain = create_agent(
    model=model,
    tools=[fetch_sales_data, calculate_revenue, generate_report],
    system_prompt="""你是数据分析助手。处理销售分析请求时：
1. 先用fetch_sales_data获取销售数据
2. 再用calculate_revenue计算收入
3. 最后用generate_report生成报告"""
)

print("【测试：多步工具调用链】")
response = agent_chain.invoke({
    "messages": [{"role": "user", "content": "分析laptop的销售情况并生成报告"}]
})

print("\n执行流程：")
step = 1
for msg in response['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"步骤 {step}: 调用 {tc['name']}({tc['args']})")
            step += 1
    elif msg.__class__.__name__ == 'ToolMessage':
        print(f"      → 结果: {msg.content}")

print(f"\n【最终报告】")
print(response['messages'][-1].content)
print()
print("说明: Agent自动按顺序调用3个工具，形成完整的数据处理流程")

【演示：工具调用链】

【测试：多步工具调用链】

执行流程：
步骤 1: 调用 fetch_sales_data({'product': 'laptop'})
      → 结果: laptop的销售量: 100台
步骤 2: 调用 calculate_revenue({'sales_data': 'laptop的销售量: 100台'})
      → 结果: 计算结果: 收入为 500,000 元
步骤 3: 调用 generate_report({'revenue_info': '收入为 500,000 元'})
      → 结果: 【销售报告】
收入为 500,000 元
报告生成时间: 2024-01-15

【最终报告】
laptop的销售收入为500,000元，已为您生成销售报告。需要我发送报告或进行其他分析吗？

说明: Agent自动按顺序调用3个工具，形成完整的数据处理流程


### 4.1 并行工具调用

In [8]:
print("【演示：并行工具调用】")
print()

# 定义独立的信息源
@tool
def get_weather(city: str) -> str:
    """获取天气信息"""
    weather_data = {"北京": "晴天 25°C", "上海": "多云 28°C", "深圳": "雨天 30°C"}
    return weather_data.get(city, "无数据")

@tool
def get_news(city: str) -> str:
    """获取新闻头条"""
    news_data = {"北京": "科技创新大会召开", "上海": "国际展览会开幕", "深圳": "新产业园落成"}
    return news_data.get(city, "无新闻")

@tool
def get_traffic(city: str) -> str:
    """获取交通状况"""
    traffic_data = {"北京": "轻度拥堵", "上海": "畅通", "深圳": "严重拥堵"}
    return traffic_data.get(city, "无数据")

# 创建Agent
agent_parallel = create_agent(
    model=model,
    tools=[get_weather, get_news, get_traffic],
    system_prompt="使用所有可用工具同时获取城市的全面信息。"
)

print("【测试：并行获取多维度信息】")
response = agent_parallel.invoke({
    "messages": [{"role": "user", "content": "给我上海的全面信息"}]
})

# 检查是否并行调用
print("\n工具调用分析：")
for msg in response['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        if len(msg.tool_calls) > 1:
            print(f"✓ 并行调用检测: 同时调用了 {len(msg.tool_calls)} 个工具")
            for tc in msg.tool_calls:
                print(f"  - {tc['name']}({tc['args']})")
        break

print(f"\n【综合信息】")
print(response['messages'][-1].content)
print()
print("说明: Agent识别到3个工具相互独立，一次性并行调用，提高效率")

【演示：并行工具调用】

【测试：并行获取多维度信息】

工具调用分析：
✓ 并行调用检测: 同时调用了 3 个工具
  - get_weather({'city': '上海'})
  - get_news({'city': '上海'})
  - get_traffic({'city': '上海'})

【综合信息】
上海的全面信息如下：

天气：多云，气温28°C。
新闻头条：国际展览会开幕。
交通状况：畅通。

如果需要更详细的信息或其他方面的内容，请告诉我。

说明: Agent识别到3个工具相互独立，一次性并行调用，提高效率


## 5. 实战项目1：智能研究助手

结合搜索、计算、数据分析等多种能力

In [9]:
print("【实战项目：智能研究助手】")
print()

import json

# 知识库
knowledge_base = {
    "Python": "Python是一种高级编程语言，由Guido van Rossum于1991年发布。",
    "LangChain": "LangChain是一个用于开发LLM应用的框架，提供了丰富的工具和组件。",
    "AI": "人工智能(AI)是计算机科学的一个分支，旨在创建能够执行智能任务的系统。"
}

# 定义工具集
@tool(response_format="content_and_artifact")
def search_knowledge(topic: str):
    """从知识库中搜索相关信息"""
    result = knowledge_base.get(topic, f"未找到关于'{topic}'的信息")
    return f"搜索结果: {result}", {"topic": topic, "found": topic in knowledge_base}

@tool
def calculate_expression(expression: str) -> str:
    """计算数学表达式，如：'2+3*4', 'pow(2, 10)'"""
    try:
        result = eval(expression, {"__builtins__": {}}, {"pow": pow, "abs": abs, "round": round})
        return f"{expression} = {result}"
    except Exception as e:
        return f"计算错误: {str(e)}"

@tool
def analyze_data(data_json: str) -> str:
    """分析JSON格式的数据，计算总和、平均值、最大最小值"""
    try:
        data = json.loads(data_json)
        if not isinstance(data, list) or not all(isinstance(x, (int, float)) for x in data):
            return "数据格式错误，需要数字列表，如: [1, 2, 3, 4, 5]"
        
        total = sum(data)
        avg = total / len(data)
        return f"数据分析:\n  总和={total}\n  平均值={avg:.2f}\n  最大值={max(data)}\n  最小值={min(data)}"
    except Exception as e:
        return f"分析错误: {str(e)}"

@tool
def save_notes(content: str) -> str:
    """保存研究笔记"""
    # 实际应用中会保存到文件或数据库
    return f"笔记已保存: '{content[:50]}...'"

# 创建带错误处理的研究助手
research_assistant = create_agent(
    model=model,
    tools=[search_knowledge, calculate_expression, analyze_data, save_notes],
    system_prompt="""你是智能研究助手，可以：
- 搜索知识库（search_knowledge）
- 进行数学计算（calculate_expression）
- 分析数据（analyze_data）
- 保存笔记（save_notes）

根据用户需求选择合适的工具组合。""",
    middleware=[
        ToolRetryMiddleware(max_retries=2, on_failure="return_message"),
        custom_error_handler
    ]
)

# 测试不同场景
test_queries = [
    "搜索LangChain的信息，然后帮我保存笔记",
    "计算 (15 + 25) * 2 的结果",
    "分析这组数据: [85, 90, 78, 92, 88]"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"【测试 {i}】: {query}")
    print('-'*60)
    
    response = research_assistant.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    
    # 显示工具调用
    for msg in response['messages']:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"→ 调用: {tc['name']}")
    
    print(f"\n答案: {response['messages'][-1].content}")

print("\n" + "="*60)
print("✓ 智能研究助手演示完成")

【实战项目：智能研究助手】


【测试 1】: 搜索LangChain的信息，然后帮我保存笔记
------------------------------------------------------------


/tmp/ipykernel_1645757/2089611902.py:61: DeprecationWarning: on_failure='return_message' is deprecated and will be removed in a future version. Use on_failure='continue' instead.
  ToolRetryMiddleware(max_retries=2, on_failure="return_message"),


✓ 工具 search_knowledge 执行成功
✓ 工具 save_notes 执行成功
→ 调用: search_knowledge
→ 调用: save_notes

答案: LangChain是一个用于开发大型语言模型（LLM）应用的框架，提供了丰富的工具和组件。我已经帮你保存了关于LangChain的相关笔记。还有其他需要帮忙的吗？

【测试 2】: 计算 (15 + 25) * 2 的结果
------------------------------------------------------------
✓ 工具 calculate_expression 执行成功
→ 调用: calculate_expression

答案: (15 + 25) * 2 的结果是 80。

【测试 3】: 分析这组数据: [85, 90, 78, 92, 88]
------------------------------------------------------------
✓ 工具 analyze_data 执行成功
→ 调用: analyze_data

答案: 这组数据的分析结果如下：
- 总和为433
- 平均值为86.60
- 最大值为92
- 最小值为78

如果需要进一步的分析或其他帮助，请告诉我。

✓ 智能研究助手演示完成


## 6. 实战项目2：数据处理Agent

模拟数据查询、转换、导出的完整流程

In [10]:
print("【实战项目：数据处理Agent】")
print()

# 模拟数据库
database = [
    {"id": 1, "name": "Alice", "department": "Engineering", "salary": 90000},
    {"id": 2, "name": "Bob", "department": "Sales", "salary": 75000},
    {"id": 3, "name": "Charlie", "department": "Engineering", "salary": 95000},
    {"id": 4, "name": "Diana", "department": "Marketing", "salary": 70000},
    {"id": 5, "name": "Eve", "department": "Engineering", "salary": 88000}
]

# 定义数据处理工具
@tool(response_format="content_and_artifact")
def query_database(department: str = None):
    """查询数据库。可选参数: department（部门名称）"""
    if department:
        results = [r for r in database if r["department"] == department]
        content = f"查询到 {len(results)} 条{department}部门的记录"
    else:
        results = database
        content = f"查询到全部 {len(results)} 条记录"
    
    return content, {"data": results, "count": len(results)}

@tool
def calculate_stats(department: str) -> str:
    """计算指定部门的统计信息（平均工资、总人数）"""
    dept_data = [r for r in database if r["department"] == department]
    if not dept_data:
        return f"{department}部门没有数据"
    
    avg_salary = sum(r["salary"] for r in dept_data) / len(dept_data)
    return f"{department}部门: {len(dept_data)}人, 平均工资={avg_salary:,.0f}元"

@tool
def export_data(format_type: str) -> str:
    """导出数据。支持的格式: csv, json"""
    if format_type == "csv":
        return "数据已导出为 data.csv (模拟)"
    elif format_type == "json":
        return "数据已导出为 data.json (模拟)"
    else:
        return f"不支持的格式: {format_type}"

# 创建数据处理Agent
data_agent = create_agent(
    model=model,
    tools=[query_database, calculate_stats, export_data],
    system_prompt="""你是数据处理助手，可以：
1. 查询数据库（query_database）
2. 计算统计信息（calculate_stats）
3. 导出数据（export_data）

按照：查询 → 分析 → 导出 的流程处理数据请求。""",
    middleware=[ToolRetryMiddleware(max_retries=2)]
)

# 测试复杂查询
print("【测试：完整数据处理流程】")
response = data_agent.invoke({
    "messages": [{"role": "user", "content": "查询Engineering部门的数据，计算统计信息，并导出为JSON格式"}]
})

print("\n执行流程：")
for msg in response['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"→ {tc['name']}({tc['args']})")

print(f"\n【最终结果】")
print(response['messages'][-1].content)
print("\n✓ 数据处理Agent演示完成")

【实战项目：数据处理Agent】

【测试：完整数据处理流程】

执行流程：
→ query_database({'department': 'Engineering'})
→ calculate_stats({'department': 'Engineering'})
→ export_data({'format_type': 'json'})

【最终结果】
Engineering部门共有3人，平均工资为91,000元。相关数据已导出为JSON格式。

✓ 数据处理Agent演示完成


## 7. 总结与最佳实践

### 核心概念回顾

**1. 错误处理策略**

| 策略 | 适用场景 | 实现方式 |
|------|---------|--------|
| 自动重试 | 网络临时故障、API限流 | `ToolRetryMiddleware` |
| 让LLM感知错误 | LLM可调整策略的错误 | 返回`ToolMessage`而非抛异常 |
| 人工介入 | 不可恢复的错误 | Human-in-the-loop |

**2. Tool Artifacts**

- **用途**：分离给LLM的文本和给应用的数据
- **配置**：`@tool(response_format="content_and_artifact")`
- **返回**：`return content, artifact`
- **访问**：`message.artifact`

**3. 工具调用模式**

- **串行**：工具按依赖顺序执行（A → B → C）
- **并行**：独立工具同时执行（[A, B, C]）
- **自适应**：根据中间结果动态决策

---

### Middleware机制

**两种拦截点：**

```python
@wrap_model_call  # 拦截模型调用
def model_middleware(request, handler):
    # 可以修改工具列表、prompt等
    return handler(request)

@wrap_tool_call  # 拦截工具执行
def tool_middleware(request, handler):
    # 可以修改参数、处理错误、记录日志
    return handler(request)
```

**常见用途：**
- 错误处理与重试
- 日志记录与监控
- 权限控制与审批
- 动态工具选择
- 性能优化（缓存等）

---

### 最佳实践

**工具设计：**
- 工具功能单一，易于理解
- 使用`response_format="content_and_artifact"`分离数据
- 提供清晰的docstring和参数类型
- 工具之间保持松耦合

**错误处理：**
- 对临时性错误使用`ToolRetryMiddleware`
- 让LLM感知可恢复的错误（返回ToolMessage）
- 对危险操作实施人工审批
- 记录所有错误日志便于调试

**性能优化：**
- 利用并行工具调用减少延迟
- 缓存常见查询结果
- 设置合理的重试次数和超时
- 使用artifact避免在content中传输大量数据

**安全性：**
- 敏感操作必须经过审批
- 验证工具参数的合法性
- 限制工具的执行权限
- 记录所有工具调用的审计日志

**监控与调试：**
- 使用middleware记录工具调用
- 监控工具执行时间和成功率
- 追踪artifact的使用情况
- 分析工具选择的准确性

---

### 常见问题

**Q1: 工具频繁失败怎么办？**
- 使用`ToolRetryMiddleware`自动重试
- 检查工具实现是否健壮
- 提供备用工具或降级方案

**Q2: 如何控制工具调用的成本？**
- 限制`recursion_limit`
- 使用缓存减少重复调用
- 在system_prompt中明确工具使用规则

**Q3: Agent不按预期顺序调用工具？**
- 在system_prompt中明确工作流程
- 改进工具的描述和命名
- 考虑工具之间的依赖关系

**Q4: artifact没有被保存？**
- 确认使用了`response_format="content_and_artifact"`
- 检查工具返回格式：`return content, artifact`
- 从`ToolMessage.artifact`而非`AIMessage`中获取

**Q5: Human-in-the-loop如何实现？**
- 简单场景：使用`wrap_tool_call`拦截
- 复杂场景：需要LangGraph的interrupt和checkpointer（后续章节）